<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex12notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex12notebookB

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

----
## 判別分析に関する補足
---

<!---
<b><font color="#ff0000">
注意:
今回の notebook の中には，コードセルを実行すると問題の解答が表示されるようになっている箇所があります．
</font>
</b>
--->

In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy の もろもろ
import scipy as sp
from scipy.stats import norm, multivariate_normal

# scikit-learn のもろもろ
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix

# NumPy の 疑似乱数生成器（rng = random number generator）
from numpy.random import default_rng

---
### 線形判別 vs 二次判別

線形判別分析では判別の境界が平面ですが，二次判別分析では二次曲面になります．二次曲面の方がクラスの境界をより柔軟に作れるため，二次判別分析の方が高い精度の予測ができる可能性があります．
線形判別分析の仮定である，クラス毎の分散共分散行列が共通という条件が成り立たないようなデータを作って，線形判別と二次判別の違いを観察してみましょう．

正規乱数で生成した次のような3クラスのデータを用います．

In [ ]:
# 疑似乱数生成器を初期化
rng = default_rng(seed=4649)
# クラスごとのデータ数，平均，分散共分散行列
N0, N1, N2 = 500, 500, 500
mu = np.array([[3, 3], [4, 7], [6.5, 3.5]])
cov = np.array([[[0.6, 0], [0, 0.6]], [[0.25, 0.2], [0.2, 0.9]], [[0.75, -0.6], [-0.6, 0.75]]])
# データを生成
X = np.empty((N0 + N1 + N2, 2))
X[:N0,        :] = rng.multivariate_normal(mu[0], cov[0], size=N0)
X[N0:N0 + N1, :] = rng.multivariate_normal(mu[1], cov[1], size=N1)
X[N0 + N1:,   :] = rng.multivariate_normal(mu[2], cov[2], size=N2)
y = np.zeros(N0 + N1 + N2, dtype=int)
y[N0:N0 + N1] = 1
y[N0 + N1:] = 2
# 散布図を描く
fig, ax = plt.subplots(figsize=(4, 4))
for k in range(3):
    ax.scatter(X[y == k, 0], X[y == k, 1], label=f'Class{k}', s=8)
ax.set_xlim(0, 9)
ax.set_ylim(0, 9)
ax.legend()
plt.show()

線形判別分析と二次判別分析を実行します．以下では，それぞれで推定された平均や分散共分散行列の値も表示するようにしてあります．

In [ ]:
# LDA
lda = LinearDiscriminantAnalysis(store_covariance=True)
lda.fit(X, y)
muL = lda.means_
covL = lda.covariance_
print('##### LDA #####')
print('(平均) = ')
print(muL)
print('(分散共分散行列) = ')
print(covL)
print()

# QDA
qda = QuadraticDiscriminantAnalysis(store_covariance=True)
qda.fit(X, y)
muQ = qda.means_
covQ = np.array(qda.covariance_)
print('##### QDA #####')
print('(平均) = ')
print(muQ)
print('(分散共分散行列) = ')
print(covQ)

線形判別では，$2\times 2$ の分散共分散行列を一つしか求めていませんが，二次判別ではクラス数ぶん求めています．判別結果を見てみると...

In [ ]:
## 散布図，確率密度の等高線，判別結果で領域を塗り分
fig, ax = plt.subplots(1, 2, figsize=(8, 4))

xmin, xmax = 0, 9
ymin, ymax = 0, 9
xx, yy = np.mgrid[xmin:xmax:0.05, ymin:ymax:0.05]
XX = np.dstack((xx, yy))

colors = ['blue', 'red', 'green']
cmap = ['Blues', 'Oranges', 'Greens']
y_predL = lda.predict(XX.reshape(-1, 2)).reshape((XX.shape[0], XX.shape[1]))
y_predQ = qda.predict(XX.reshape(-1, 2)).reshape((XX.shape[0], XX.shape[1]))

for k in range(3):
    # LDA
    ax[0].scatter(X[y == k, 0], X[y == k, 1], label=f'Class{k}', s=8)
    zz = multivariate_normal.pdf(XX, mean=muL[k], cov=covL)
    ax[0].contour(xx, yy, zz, colors=colors[k])
    ax[0].contourf(xx, yy, (y_predL == k).astype(int), cmap=cmap[k], alpha=0.2)
    # QDA
    ax[1].scatter(X[y == k, 0], X[y == k, 1], label=f'Class{k}', s=8)
    zz = multivariate_normal.pdf(XX, mean=muQ[k], cov=covQ[k])
    ax[1].contour(xx, yy, zz, colors=colors[k])
    ax[1].contourf(xx, yy, (y_predQ == k).astype(int), cmap=cmap[k], alpha=0.2)

ax[0].set_xlim(xmin, xmax)
ax[0].set_ylim(ymin, ymax)
ax[0].set_title(f'LDA (accuracy = {lda.score(X, y):.3f})')
ax[1].set_xlim(xmin, xmax)
ax[1].set_ylim(ymin, ymax)
ax[1].set_title(f'QDA (accuracy = {qda.score(X, y):.3f})')
plt.show()

左の線形判別ではクラスの境界が直線ですが，右の二次判別では二次曲線になっており，二次判別の方がaccuracy（全てのデータのうちクラスを正しく予測できたものの割合）も高くなっていることが分かります．

---
### 二次判別でうまくいかないこともある

上記のことからすると，コンピュータを使うので計算の手間をあまり惜しむ必要がない場合，常に二次判別を使えばよいようにも思えます．しかし，実際には，次のようなことに注意が必要です．

- 二次判別の方が推定すべきパラメータの数が多い： 分散共分散行列ひとつは $\frac{1}{2}D(D+1)$ 個のパラメータを持ちます．二次判別の場合，これをクラスごとに推定しますので，パラメータ数がさらに $K$ 倍になります．サンプルサイズの小さいデータだと分散共分散行列の推定がうまくできず，良い結果が得られないことがあります．
- 正規分布の仮定が成り立たない場合，線形判別でも二次判別でも良い結果は得られないことがある： 線形判別も二次判別も，クラスごとのデータが正規分布に従うことを仮定しています．この仮定が満たされないような場合には，どちらの手法でも良い結果が得られないことがあります．

次の例は，データの分布が明らかに正規分布ではないような場合にどうなるかを示しています．

In [ ]:
### 正規分布に従わない人工データ
from sklearn.datasets import make_moons
X_moon, Y_moon = make_moons(n_samples=300, noise=0.2)

### QDA してみる
qda = QuadraticDiscriminantAnalysis(store_covariance=True)
qda.fit(X_moon, Y_moon)

### 散布図と判別結果
fig, ax = plt.subplots(1, 2, figsize=(8, 4))

## 確率密度と判別結果の描画のためのグリッドデータの準備
xmin, xmax = -1.5, 2.5
ymin, ymax = -1.0, 1.5
xx, yy = np.mgrid[xmin:xmax:0.01, ymin:ymax:0.011]
XX = np.dstack((xx, yy))

##  散布図
for i in [0, 1]:
    ax[i].scatter(X_moon[Y_moon == 1, 0], X_moon[Y_moon == 1, 1], label='Class1', s=10)
    ax[i].scatter(X_moon[Y_moon == 0, 0], X_moon[Y_moon == 0, 1], label='Class0', s=10)
    ax[i].set_xlim(xmin, xmax)
    ax[i].set_ylim(ymin, ymax)
    ax[i].set_aspect('equal')
    ax[i].legend()

## 確率密度の等高線
zz = multivariate_normal.pdf(XX, mean=qda.means_[1], cov=qda.covariance_[1])
ax[1].contour(xx, yy, zz, colors='blue')
zz = multivariate_normal.pdf(XX, mean=qda.means_[0], cov=qda.covariance_[0])
ax[1].contour(xx, yy, zz, colors='red')

## 判別結果で領域を塗り分ける
zz = qda.predict(XX.reshape(-1, 2)).reshape((XX.shape[0], XX.shape[1]))
ax[1].contourf(xx, yy, zz,   cmap='Blues',   alpha=0.2)
ax[1].contourf(xx, yy, 1-zz, cmap='Oranges', alpha=0.2)

plt.show()

当たり前ですが，二次判別分析でもうまく判別できていません．

「機械学習I/II」では，このような場合でもうまく判別できるような手法を取り上げます．

---
### データの標準化

「主成分分析(2)」の回の「主成分分析とデータの標準化」という節で，データの標準化が主成分分析の結果に与える影響について考えました．
データによっては変数間で値の範囲・大きさ範囲が全く異なることがあるので，データを標準化してから分析した方がよい場合がある，ということでした．実例は示しませんが，判別分析においても同様のことが言えます．

---
### クラスごとのデータの出現確率の扱い

ここまで解説してきた判別分析の方法は，クラスごとの正規分布を推定し，それらの対数尤度を基準として判別するというものです．「クラスごとのデータが正規分布に従う」という仮定をおいており，その仮定が成り立たないときにはうまくいかないかもしれない，ということはすでに説明しました．

実は，ここまで説明してきた方法には，隠れた仮定がもうひとつあります．それは，「どのクラスのデータも等しい確率で生起する」というものです．この仮定が成り立たない場合，すなわち，クラスごとのデータの出現確率に偏りがあるような場合，クラスごとの正規分布に対する対数尤度を基準とする，という方法ではよい結果が得られない可能性があります．

これに対処する方法の一つが，クラスごとのデータの出現確率（これを「クラス事前確率」といいます）まで含めてモデル化を行う，というものです．
具体的な定式化等については，「機械学習I/II」で取り上げます．scikit-learn の判別分析の実装では，このクラス事前確率を含めたモデルが使われており，データ中の各クラスの出現頻度からその値を推定するようになっています（注）．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※ 注意: 以下のリンク先のドキュメントの $P(y=k)$ というのが事前確率です：
<a href="https://scikit-learn.org/stable/modules/lda_qda.html#mathematical-formulation-of-the-lda-and-qda-classifiers">
https://scikit-learn.org/stable/modules/lda_qda.html#mathematical-formulation-of-the-lda-and-qda-classifiers
</a>
</span>